In [ ]:
!pip install feedparser
!pip install newspaper3k
!pip install konlpy
!pip install bs4
import feedparser
from newspaper import Article
from konlpy.tag import Okt
from bs4 import BeautifulSoup
import numpy as np
from collections import Counter
from operator import eq
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
urls = ["http://rss.etnews.com/Section901.xml", 
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml", 
        "http://rss.etnews.com/Section904.xml"]

def crawl_rss(urls):
  array_rss = []
  link_rss = set()
  for url in urls:
    print("[Crawl RSS]", url)
    parse_rss = feedparser.parse(url)
    for p in parse_rss.entries:
      if p.link not in link_rss:
        array_rss.append({'title':p.title, 'link':p.link})
        link_rss.add(p.link)
      else:
        print("중복된 제목 :", p.title) 
  return array_rss

# [단계#1] 모든 RSS파일(xml형식)을 돌아다니며 기사의 제목/link 추출
list_articles = crawl_rss(urls)
print(list_articles[0]) # 첫번째 기사 출력

[Crawl RSS]  http://rss.etnews.com/Section901.xml
[Crawl RSS]  http://rss.etnews.com/Section902.xml
[Crawl RSS]  http://rss.etnews.com/Section903.xml
중복된 제목 : 삼성전자·삼성디스플레이 '마이크로 OLED'로 뭉친다
중복된 제목 : KT 투자시계 '스톱'… ICT 생태계 초비상
중복된 제목 : PC 금융보안인증 SW, 北 해킹 루트 활용
중복된 제목 : 글로벌 해킹 조직 락빗 "국세청 모든 정보, 4월 1일 공개할 것"
중복된 제목 : 상반기 공공 최대 클라우드 사업, 삼성SDS가 수주
[Crawl RSS]  http://rss.etnews.com/Section904.xml
중복된 제목 : '생물학적 자녀만 550명'…정자 너무 많이 기증한 네덜란드 남성
중복된 제목 : 이스라엘서 6000년전 대형 낚싯바늘 발굴..."상어잡이용"
중복된 제목 : 러시아 도심 하늘에 나타난 정체불명 '검은 고리'
중복된 제목 : 데이트 후 사라진 1억3000만원...아르헨 '검은 과부' 주의보
중복된 제목 : 'K-칩스법' 국회 본회의 통과…선거제 개편 전원위도 첫발
중복된 제목 : 상반기 공공 최대 클라우드 사업, 삼성SDS가 수주
중복된 제목 : 글로벌 해킹 조직 락빗 "국세청 모든 정보, 4월 1일 공개할 것"
중복된 제목 : [Bett 2023] AI 활용부터 안전·보안까지…교육 디지털 혁신 '가속'
중복된 제목 : 동운아나텍, 메타 벤더 등록...XR 시장 진출 교두보 마련
중복된 제목 : 尹, USTR 대표 접견 "반도체법·IRA 우리 기업 배려 해달라"
중복된 제목 : 'SMP 상한제' 내달 재시행 조건 충족…업계 "전력수요 적어 효과 없다" 반발
중복된 제목 : 코웨이 매트리스 '나홀로 성장'
중복된 제목 : 개인투자조합 요건 강화에 액셀러레이터 울상
중복된 제목 : 바디프랜드 '허리 통증 완화 의료기기' 홈 헬스케어 시장 잡는다
중복된 제목 

In [ ]:
def crawl_article(url, language='ko'):
  print("[Crawl Article] ", url)
  a = Article(url, language = language)
  a.download()
  a.parse()
  text = BeautifulSoup(a.text,"html5lib").get_text
  return a.title, a.text

def preprocessing(text):
  text_artivle = BeautifulSoup(text).get_text()
  return text_article


# [단계#2] 모든 link를 돌아다니며 본문text 추출
for article in list_articles:
  _, text = crawl_article(article['link'])
  article['text'] = text
print(list_articles[0]) # 첫번째 기사 출력

[Crawl Article]  https://www.etnews.com/20230330000326
[Crawl Article]  https://www.etnews.com/20230330000309
[Crawl Article]  https://www.etnews.com/20230330000283
[Crawl Article]  https://www.etnews.com/20230330000244
[Crawl Article]  https://www.etnews.com/20230330000316
[Crawl Article]  https://www.etnews.com/20230330000323
[Crawl Article]  https://www.etnews.com/20230330000167
[Crawl Article]  https://www.etnews.com/20230330000143
[Crawl Article]  https://www.etnews.com/20230330000290
[Crawl Article]  https://www.etnews.com/20230330000241
[Crawl Article]  https://www.etnews.com/20230330000193
[Crawl Article]  https://www.etnews.com/20230330000095
[Crawl Article]  https://www.etnews.com/20230330000086
[Crawl Article]  https://www.etnews.com/20230330000069
[Crawl Article]  https://www.etnews.com/20230330000287
[Crawl Article]  https://www.etnews.com/20230330000282
[Crawl Article]  https://www.etnews.com/20230330000277
[Crawl Article]  https://www.etnews.com/20230330000259
[Crawl Art

In [ ]:
# 리스트에 있는 모든 기사를 돌아다니며teXt를 뽑는다.
text_articles = [article['text'] for article in list_articles]
 #TF-IDF를 근거로 주어진 문장을 벡터로 만드는 객체를 생성한다.

encoder = TfidfVectorizer(max_features = 5000)
#fit_trans로 5000사이즈 벡터로 기사변환
# 아래 tfidf_matrix는 이러한 모든 기사를 5000사이즈 벡터로 변환한 결과르 ㄹ보관


matrix_vectors = encoder.fit_transform(text_articles) # nㄱ애의 중복되지 않은 기사가 검색되었고 각 기사는 5천 사이즈 벡터로 변환

print(matrix_vectors.shape) # 첫번쨰 기사의 벡터

#첫번쨰 기사의 벡터와 다른 기사들을 다 비교한것
#i는 아래 프린트에서 비교하는 문서들을 순자로 번호표 여주기 위함
for i, vector in enumerate(tfidf_matrix[1:]):
  print(f"[1] and [{i+2 : 3d}]",   cosine_similarity(tfidf_matrix[0], vector)[0][0]
)

(75, 5000)
[1] and [  2] 0.032009588859119795
[1] and [  3] 0.05607924498085781
[1] and [  4] 0.05257951656857042
[1] and [  5] 0.005057998626586577
[1] and [  6] 0.010173519147610027
[1] and [  7] 0.02510695824450663
[1] and [  8] 0.030722137415140162
[1] and [  9] 0.011933954679267224
[1] and [ 10] 0.015277174188271585
[1] and [ 11] 0.005612010721338257
[1] and [ 12] 0.0056644129046919825
[1] and [ 13] 0.02251231699137698
[1] and [ 14] 0.002127521942839256
[1] and [ 15] 0.013810897479235073
[1] and [ 16] 0.010645777551591878
[1] and [ 17] 0.008280546922487544
[1] and [ 18] 0.024034025627722624
[1] and [ 19] 0.010974831553611225
[1] and [ 20] 0.009181077737524987
[1] and [ 21] 0.01124067576811858
[1] and [ 22] 0.011994766049921831
[1] and [ 23] 0.013464867636143877
[1] and [ 24] 0.006898663452062432
[1] and [ 25] 0.02966874637253883
[1] and [ 26] 0.012355457373947809
[1] and [ 27] 0.018521061908199892
[1] and [ 28] 0.007459536196418395
[1] and [ 29] 0.05151525188738793
[1] and [ 30] 0

In [ ]:
# # [단계#3] 모든 본문text에서 명사 & (키워드, 빈도수) 추출

# def get_keywords(text, nKeywords=10): #키워드ㅜ 추출함수, 디폴트로 10개의 키워드 추출 지정
#   spliter = Okt() # konlpy에 의해서 문장을 형태소 별로 쪼갬
#   nouns = spliter.nouns(text) #spliter에서 nouns함수를 불러서 텍스트를 넣으면 그 text의 명사만 추출
#   count = Counter(nouns)  #추출된 명사들의 출현ㅇ빈도 추출
#   list_keywords = []
#   for n, c in count.most_common(nKeywords):#가장 출현빈도가 높은 명사부터 순차적으로 10번출력
#     item = {'keyword': n, 'count': c} #출력할 떄는 {'keyword' , 'count'} 형식으로 출력
#     list_keywords.append(item) #출력된 item들을 list_keywords에 저장
#   return list_keywords #10회 반복이 끝나면 이를 list_ketwords에저장

# for article in list_articles: #모든기사를 돌아다니며 text에서 명사를 추출하고 빈도에 따라 키워드 / 빈도 ㅊ출
#   keywords = get_keywords(article['text']) #text를 get_keywords함수에ㅐ 넣고 결과를 그keyword와 coount로 받음(10걔)
#   article['keywords'] = keywords #위의 함수 결과값인 키워드?빈도의 쌍을 KEYWORds로 만들어저장
# print(list_articles[0]) # 첫번째 기사 출력

{'title': "코리아모빌리티, 세계 최초 허브리스 전기자전거 개발…글로벌 무대 '씽씽'", 'link': 'https://www.etnews.com/20230330000102', 'text': "스마트 모빌리티 스타트업 코리아모빌리티(대표 박정석)는 5월부터 3년간 총 1만5000여대의 바큇살 없는 미래형 허브리스 전기자전거 '코모 바이크(Komo Bike)'를 유럽에 수출할 계획이라고 30일 밝혔다.\n\n이 회사가 세계 최초로 개발한 코모 바이크는 바큇살이 있는 전기자전거와는 달리 휠 중앙이 뚫려 있다. 기존 전기자전거에 비해 무게중심이 낮아 주행 안정성이 높고 자체 개발한 헤어핀 모터와 인버터를 재택해 에너지 효율이 뛰어나다. 본딩 공법으로 제조해 기존 용접 대비 강도를 3배 이상 높였다. 주행거리도 늘어 1번 충전으로 약 100㎞ 거리를 이동할 수 있다. 국산 슈퍼커패시터 배터리를 적용해 30분만에 완충이 가능하다.\n\n\n\n코리아모빌리티는 유럽 수출을 위해 코모 바이크 유럽인증(CE)을 획득했다. 허브리스 타입 전기자전거 제품으로 CE인증을 받은 것도 세계에서 처음이다. 지난해 10월에는 전 세계 12개국에 지사와 법인을 운영 중인 STX와 3년 총판 계약을 맺고, 연간 5000여대 전기자전거를 유럽에 수출할 예정이다.\n\n\n\n박정석 대표는 “유럽은 자전거가 생활화돼 전기자전거가 자동차처럼 등록 관리된다”며 “유럽인들은 튼튼하면서 주행거리가 긴 친환경 전기자전거를 많이 찾는다”고 말했다.\n\n\n\n코리아모빌리티는 전기 자전거에 이어 전기오토바이 시장에도 출사표를 냈다. 최대 시속 90㎞에 주행거리 약 70㎞ 정도의 '코모 모터사이클(Komo Motorcycle)'을 출시했다. 자체 개발한 이륜차용 '헤어핀 모터'를 사용해 모터 효율을 극대했다.\n\n\n\n대부분 오토바이 차량에서 사용하는 허브모터는 모터 효율이 70% 후반에서 80% 초·중반 사이다. 코리아모빌리티가 개발한 '헤어핀 모터'는 정교한 구조 설계로 96% 이상의 효율을

In [ ]:
# # [단계#4] 검색어를 입력받아서 그 검색어를 가지고 있는 기사 출력
# query = input()
# def search_articles(query, list_keywords): # 쿼리가 키워드 리스트에 있으면 횟수출력
#   nWords = 0 # 아래의 if문에 걸리지 않으면(즉, 쿼리가 키워드에 없음) 0
#   for kw in list_keywords: # 키워드 리스트 검색
#     noun = kw['keyword']  #쿼리와 동일한 키워드가 존재한다면
#     count = kw['count'] #nWords는 그 회쉬(count)로ㅓ 갱신
#     if eq(query, noun): # 쿼리와 동일한 키워드가 존재한다면
#       nWords = count # nWords는 그 회수(count)로 갱신 
#   return nWords #결과적으로 쿼리의 출현회수를 출력
# for article in list_articles: #모든 기사를 돌아다니며 입력받은 쿼리가 몇 번 나오는지 검색(쿼리가 키워드일경우에만)
#   nQuery = search_articles(query, article['keywords'])
#   if nQuery != 0: # 쿼리가 키워드에 존재하지 않은 문서는 출력하지 않음
#     print('[TF]:', nQuery, '[Title]:', article['title'], '[URL]:', article['link'])

존
[TF]: 12 [Title]: 지옥에서 온 애견인 '존 윅 4', 말은 더 줄이고 총은 더 쏜다 [URL]: https://www.etnews.com/20230330000009
[TF]: 12 [Title]: 지옥에서 온 애견인 '존 윅 4', 말은 더 줄이고 총은 더 쏜다 [URL]: https://www.etnews.com/20230330000009
